In [1]:
# standart librairies
import os,sys
import numpy as np
import xarray as xr
import pandas as pd
import datetime

# plot
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib import ticker

from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [2]:
CB_colors = {
	'blue':	'#377eb8',
	'orange':  '#ff7f00',
	'green':   '#4daf4a',
	'pink':	'#f781bf',
	'brown':   '#a65628',
	'purple':  '#984ea3',
	'gray':	'#999999',
	'red': 	'#e41a1c',
	'yellow':  '#dede00'
}

In [3]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38369 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:38369/status,
Dashboard: http://127.0.0.1:38369/status,Workers: 5
Total threads: 10,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37469,Workers: 5
Dashboard: http://127.0.0.1:38369/status,Total threads: 10
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:45933,Total threads: 2
Dashboard: http://127.0.0.1:42073/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:32781,


# loading CPOM arctic data

In [4]:
# first manipulations of the dataset for ease: coordinates
chunk_size = {'lat':25,'lon':80}
diri = '/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/'
namo = 'Full_DOT_data_Arco.nc'
CPOMdat = xr.open_dataset(diri+namo,chunks=chunk_size)
# 2011-01 to 2020-12: change time unit
time = pd.date_range(start='01/2011', end='01/2021', freq='M')
CPOMdata = CPOMdat.rename({'time':'time_counter'})
CPOMnav_lat = CPOMdata.lats
CPOMnav_lon = CPOMdata.lons
CPOMdata = CPOMdata.assign_coords(nav_lon=CPOMnav_lon, nav_lat=CPOMnav_lat, time_counter=time)
sla = CPOMdata.Sea_level_anomaly

# Apply the BGRmask and do the time series on the box

In [70]:
BGRmaskCPOM = xr.open_dataset('/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/Full_DOT_data_Arco_BGRmaskcorr.nc').BGRmaskCPOM.squeeze()

sla_masked = sla.where(BGRmaskCPOM,drop=True).sel(time_counter=slice('2011','2018'))
sla_masked_1y = sla_masked.groupby('time_counter.year').mean('time_counter')

In [71]:
xdist = CPOMdata.xdist
ydist = CPOMdata.ydist
cellarea = xdist*ydist
cellareaBGr = cellarea.where(BGRmaskCPOM,drop=True)
totalOceanArea = cellareaBGr.sum()
WeightBGR_CPOM = (cellareaBGr/totalOceanArea) #.compute()

In [77]:
# weighted mean in the BGRbox
sla_boxmasked_mean_1m = (sla_masked*WeightBGR_CPOM).sum(('x','y'))
sla_boxmasked_mean_1m = sla_boxmasked_mean_1m - sla_boxmasked_mean_1m.mean()

sla_boxmasked_mean_1y = (sla_masked_1y*WeightBGR_CPOM).sum(('x','y'))
sla_boxmasked_mean_1y = sla_boxmasked_mean_1y - sla_boxmasked_mean_1y.mean()


In [84]:
ds_ts_sla_box = sla_boxmasked_mean_1m.to_dataset(name='ts_sla_box')

In [86]:
ds_ts_sla_box.to_netcdf("/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/Full_DOT_data_Arco_ts_BGRbox.nc", mode='w') 